<a href="https://colab.research.google.com/github/deek2689/CERC_AI/blob/main/CERC_Commenting_Predictions_NO_FineTune_Gemini_Safety_low.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade google-cloud-aiplatform


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.1 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.71.1
    Uninstalling google-cloud-aiplatform-1.71.1:
      Successfully uninstalled google-cloud-aiplatform-1.71.1


In [2]:
!pip install google-cloud-storage

In [3]:
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [4]:
import base64
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting


In [38]:
from google.cloud import storage
import json
## IMPORTING TEST DATA FROM GCS BUCKET
def load_test_data_from_gcs(bucket_name, blob_name):
    """Load JSONL data from a Google Cloud Storage bucket."""
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    data = []
    for line in blob.download_as_text().splitlines():
        data.append(json.loads(line))
    return data

bucket_name = 'cerc-h'
blob_name = 'SafeCity Data/CERC_SafeCity_Commenting_test.jsonl'
test_data = load_test_data_from_gcs(bucket_name, blob_name)

In [39]:
len(test_data)

1701

In [40]:
test_data[104]

{'messages': [{'role': 'system',
   'content': "Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."},
  {'role': 'user',
   'content': 'STATEMENT: We do not have toilet in our house. We usually access the public toilet complex. The toilet is usually surrounded by a group of boys and they pass comments.It happened at all hours of the day.'},
  {'role': 'model', 'content': True}]}

In [41]:
test_data[0]['messages'][-1]['content']

True

In [42]:
test_data[0]['messages'][1]['content']

'STATEMENT: During morning, a woman was walking by and thin guy came around and called her names , and commented on her figure . The woman handled him and abused .'

In [43]:
len(test_data)

1701

In [44]:
import time
from tqdm import tqdm
import vertexai
from vertexai.generative_models import GenerativeModel, SafetySetting


def generate_all_conversations(test_data):
    # Initialize Vertex AI with project details
    vertexai.init(project="conv-derailment", location="us-central1")
    textsi_1 = """Classify if the following statement falls under commenting related to sexual harassment. The output must be a single label: 'True' or 'False'."""
    model = GenerativeModel(
        "gemini-1.5-pro-002",
        system_instruction=[textsi_1]
    )

    # Generation configuration
    generation_config = {
        "max_output_tokens": 8192,
        "temperature": 0,
        "top_p": 0,
    }

    # Safety settings configuration
    safety_settings = [
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
            threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
        ),
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
            threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
        ),
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
            threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
        ),
        SafetySetting(
            category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
            threshold=SafetySetting.HarmBlockThreshold.BLOCK_ONLY_HIGH
        ),
    ]
    results = []
    for data in tqdm(test_data, desc="Making Predictions"):
        text1 = data['messages'][1]['content']  # Extract the conversation text
        try:
            # 1 request per second = 60 per minute
            time.sleep(1)

            # Generate content and handle streaming results
            responses = model.generate_content(
                [text1],
                generation_config=generation_config,
                safety_settings=safety_settings,
                stream=True,
            )

            # Collect the streamed response parts
            conversation_result = ""
            for response in responses:
                if response and hasattr(response, 'text'):
                    # Append only non-empty text
                    if response.text.strip():
                        conversation_result += response.text.strip()

            # Append the complete conversation result to the results list
            results.append(conversation_result if conversation_result else "No response")
        except Exception as e:
            print(f"Error processing conversation: {e}")
            results.append("Error")

    return results

# Run the generation for all conversations
results = generate_all_conversations(test_data)

# Output or further process the results
for result in results:
    print(result)

Making Predictions:   2%|▏         | 35/1701 [01:01<52:08,  1.88s/it]

Error processing conversation: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "PROHIBITED_CONTENT",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.3433423,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.25832576
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.06754686,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.117572024
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.45786405,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.3584656
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "prob

Making Predictions:   6%|▌         | 97/1701 [02:51<39:57,  1.49s/it]

Error processing conversation: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "PROHIBITED_CONTENT",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.26284194,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.25238422
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.08151978,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.120025486
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.4941102,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.3372038
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "prob

Making Predictions:  11%|█         | 189/1701 [05:43<57:12,  2.27s/it]

...

Error processing conversation: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "PROHIBITED_CONTENT",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "LOW",
      "probability_score": 0.47828522,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.3276681
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.09401007,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.13477601
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.4205999,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.29747003
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "probability"

Making Predictions:  99%|█████████▊| 1678/1701 [54:52<00:48,  2.12s/it]

Error processing conversation: Cannot get the response text.
Cannot get the Candidate text.
Response candidate content has no parts (and thus no text). The candidate is likely blocked by the safety filters.
Content:
{}
Candidate:
{
  "finish_reason": "PROHIBITED_CONTENT",
  "safety_ratings": [
    {
      "category": "HARM_CATEGORY_HATE_SPEECH",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.33372065,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.22405581
    },
    {
      "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
      "probability": "NEGLIGIBLE",
      "probability_score": 0.06656918,
      "severity": "HARM_SEVERITY_NEGLIGIBLE",
      "severity_score": 0.15203233
    },
    {
      "category": "HARM_CATEGORY_HARASSMENT",
      "probability": "LOW",
      "probability_score": 0.54431736,
      "severity": "HARM_SEVERITY_LOW",
      "severity_score": 0.31997126
    },
    {
      "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
      "probabilit

Making Predictions: 100%|██████████| 1701/1701 [55:40<00:00,  1.96s/it]


True
True
False
True
True
True
True
False
True
True
True
True
True
True
False
False
True
True
True
True
False
True
False
True
True
True
True
True
True
True
True
True
True
True
Error
True
True
True
True
True
True
False
False
True
True
False
True
True
False
True
True
False
True
True
True
True
True
True
True
False
True
True
False
True
False
True
True
False
True
True
True
True
True
True
False
True
True
False
True
True
True
True
False
True
False
True
False
True
True
True
True
True
True
True
True
True
Error
True
True
True
True
False
True
False
True
False
True
True
True
True
True
True
False
True
True
True
True
True
False
True
True
False
True
True
True
True
True
True
True
True
True
True
False
False
True
True
False
True
True
False
True
False
False
True
True
True
True
True
True
True
False
True
True
True
True
True
True
True
True
True
True
True
False
True
True
True
True
True
True
False
True
True
True
False
True
False
True
True
True
False
True
True
False
True
True
True
True
True
Error
True
True
Tru

In [45]:
print(results)

['True', 'True', 'False', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'False', 'True', 'True', 'True', 'True', 'False', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'Error', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'False', 'True', 'True', 'False', 'True', 'True', 'False', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'False', 'True', 'False', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'False', 'True', 'False', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'True', 'Error', 'True', 'True', 'True', 'True', 'False', 'True', 'False', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'True', 'True', 'True', 'False', 'True', 'True', 'Fal

In [47]:
predictions = results

In [48]:
def extract_true_labels(data):
    true_labels = []

    for item in test_data:
        # Extract the model's response, which is the true label
        model_label = item['messages'][-1]['content']
        true_labels.append(model_label)

    return true_labels

true_labels = extract_true_labels(test_data)


In [50]:
len(true_labels)

1701

In [51]:
len(predictions)

1701

In [53]:
type(true_labels[1])

bool

In [54]:
# convert true_labels in df from boolean to string

true_labels_str = [str(label) for label in true_labels]


In [55]:
import ast
import pandas as pd

df = pd.DataFrame({
    'Predictions': predictions,
    'True Labels': true_labels_str
})

In [56]:
# Filter rows where 'True Labels' are not 'True' or 'False'
filtered_df = df[~df['Predictions'].isin(['True', 'False'])]

# Print the rows that do not match 'True' or 'False'
print(filtered_df)

     Predictions True Labels
34         Error       False
96         Error       False
188        Error        True
248        Error       False
298        Error       False
362        Error       False
433        Error        True
467        Error       False
709        Error        True
821        Error       False
858        Error       False
902        Error       False
908        Error       False
941        Error       False
1123       Error       False
1150       Error        True
1251       Error       False
1383       Error       False
1458       Error       False
1473       Error       False
1481       Error       False
1496       Error       False
1547       Error        True
1559       Error       False
1566       Error       False
1579       Error       False
1590       Error       False
1677       Error       False


In [57]:
# Find missing values in 'preds' column
missing_values = df[df['Predictions'].isna()]

# Display missing values with their index
print(missing_values)

Empty DataFrame
Columns: [Predictions, True Labels]
Index: []


In [58]:
# df = df.replace('LABEL: False', 'False')
# df = df.replace('LABEL: True', 'True')
df = df.replace('Error', 'True')

In [60]:
# Filter rows where 'True Labels' are not 'True' or 'False'
filtered_df = df[~df['Predictions'].isin(['True', 'False'])]

# Print the rows that do not match 'True' or 'False'
print(filtered_df)

Empty DataFrame
Columns: [Predictions, True Labels]
Index: []


In [61]:
def num_label(label):
    if label == 'True':
        return 0
    elif label == 'False':
        return 1
    else:
        return label

In [62]:
df = df.applymap(num_label)

<ipython-input-62-e43f4bb705b2>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(num_label)


In [63]:
df

,Predictions,True Labels
0,0,0
1,0,1
2,1,1
3,0,1
4,0,1
...,...,...
1696,0,0
1697,0,0
1698,1,1
1699,0,1


In [64]:
actual_labels = df['True Labels']
predicted_labels = df['Predictions']

In [65]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report

In [66]:
## Regular

accuracy = accuracy_score(actual_labels, predicted_labels)
precision = precision_score(actual_labels, predicted_labels)
recall = recall_score(actual_labels, predicted_labels)
F1_score = f1_score(actual_labels, predicted_labels)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"f1-Score: {F1_score}")

Accuracy: 0.442092886537331
Precision: 0.5949612403100775
Recall: 0.2932187201528176
f1-Score: 0.39283429302623163


In [67]:
#macro

precision_macro = precision_score(actual_labels, predicted_labels, average='macro')
recall_macro = recall_score(actual_labels, predicted_labels, average='macro')
f1_score_macro = f1_score(actual_labels, predicted_labels, average='macro')

# Print results

print(f"Precision_macro: {precision_macro}")
print(f"Recall_macro: {recall_macro}")
print(f"f1-Score_macro: {f1_score_macro}")

Precision_macro: 0.48524433323520755
Recall_macro: 0.48682342735469625
f1-Score_macro: 0.43839648310909185


In [ ]:
classification_report = classification_report(actual_labels, predicted_labels)
print(classification_report)

              precision    recall  f1-score   support

           0       0.66      0.48      0.55       684
           1       0.59      0.75      0.66       684

    accuracy                           0.61      1368
   macro avg       0.62      0.61      0.61      1368
weighted avg       0.62      0.61      0.61      1368

